In [0]:
%run ../utils/common

In [0]:
dbutils.widgets.text("environment", "", "")
environment = dbutils.widgets.get("environment")

In [0]:
catalog_name = settings[environment]['catalog_name']


In [0]:
spark.sql(f"""
          
CREATE OR REPLACE TABLE {catalog_name}.udp_wcm_gold_vcm_dwh.d_sale_price_group
TBLPROPERTIES (
  'DELTA.AUTOOPTIMIZE.OPTIMIZEWRITE' = 'TRUE',
  'DELTA.AUTOOPTIMIZE.AUTOCOMPACT' = 'TRUE'
)
AS
WITH ranked_data AS (
  SELECT
    kappl AS app,
    kschl AS cond_type,
    vkorg AS sale_org,
    vtweg AS distribution_channel,
    konda AS price_group,
    matnr AS product_id,
    vrkme AS sale_unit,
    kfrst AS status_release,
    datbi AS valid_to,
    recordmode AS record_mode,
    datab AS valid_from,
    kbstat AS status_processing,
    knumh AS cond_no,
    ROW_NUMBER() OVER (PARTITION BY kappl, kschl, vkorg, vtweg, konda, matnr, vrkme, kfrst, datbi ORDER BY proc_date) AS row_num
  FROM  {catalog_name}.udp_wcm_silver_sap_bw.sap_vccmmrb28
  WHERE proc_date = (SELECT MAX(proc_date) FROM  {catalog_name}.udp_wcm_silver_sap_bw.sap_vccmmrb28)
)
SELECT
  app,
  cond_type,
  sale_org,
  distribution_channel,
  price_group,
  product_id,
  sale_unit,
  status_release,
  valid_to,
  record_mode,
  valid_from,
  status_processing,
  cond_no
FROM ranked_data
WHERE row_num = 1;

          """)